<a href="https://colab.research.google.com/github/argetan/TIL/blob/master/2021_10_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 패키지 및 종속성 설치

# 패키지 로드
import pandas as pd                                                           # 분석
import numpy as np                                                            # 분석
from sklearn.neighbors import KNeighborsClassifier                            # KNN
from sklearn.model_selection import cross_val_score, KFold, train_test_split  # 모델 평가
import multiprocessing as mp                                                  # 멀티프로세싱
import glob, os, re                                                           # 공통
import datetime as dt
import plotly.express as px                                                   # 시각화 - Plotly
import plotly.io as pio                                                       # 시각화 - Plotly
import folium

# 전역 설정
pio.templates.default = "plotly_dark"                                         # Plotly 테마

In [ ]:
# 사용자 클래스 - 공통
class CustomUtils:
    """
    자주 사용되는 메소드를 담은 사용자 클래스입니다.
    """
    
    @classmethod
    def display_htmls(_cls, *args, orientation='h'):
        """복수의 html을 한 행에 출력합니다."""
        from IPython.display import display_html

        display_style = orientation=='h' and 'flex' or 'block'
        html_str = f'<div style="display:{display_style};justify-content:space-around;align-items:center;">'
        for html_src in args:
            html_str += '<div>' + html_src + '</div>'
        html_str += '</div>'
        display_html(html_str, raw=True)
    
    @classmethod
    def jpeg_binary_to_img_src(_cls, jpeg_binary=None):
        """바이너리 타입의 이미지 데이터를 받아 img tag롤 반환합니다."""
        from binascii import b2a_base64
        
        b64_data = b2a_base64(jpeg_binary).decode('ascii')
        return f"<img src='data:image/jpeg;base64,{b64_data}'>"

In [ ]:
### 데이터 로드 및 전처리
coldtypes = {'project_name': 'category', 'station': 'category', '계절': 'category', '구간': 'category',
             '너울등급_code': str, '너울방향_code': str, '수심(text)': str, '날씨_code': str, '수색_code': str,
             '연도': 'int16', '운량_code': 'category', '운형_code': str, '월': 'category', '파도등급_code': str,
             '파향_code': str, '풍력등급_code': str, '풍향_code': str, '해역': 'category'}
colnames = {'yyyy-mm-dd hh:mm:ss': '일시', 'Longitude [degrees_east]': '경도', 'Latitude [degrees_north]': '위도'}

def predict_by_knn(x_tr, y_tr, x_te):
    x_train, x_valid, y_train, y_valid = train_test_split(x_tr, y_tr, test_size=0.2)
    # 최적의 n을 구하기
    k_fold = KFold(n_splits=10, shuffle=True)
    cross_validation_scores = []
    for n in range(3, 10, 2):
        knn = KNeighborsClassifier(n_neighbors=n)
        scores = cross_val_score(knn, x_train, y_train, cv=k_fold, scoring='accuracy')
        cross_validation_scores.append((n, scores.mean()))
    n_best = max(cross_validation_scores, key=lambda x: x[-1])[0]
    # KNN 모델링
    knn = KNeighborsClassifier(n_neighbors=n_best)
    knn.fit(x_tr, y_tr)
    return knn.predict(x_te)

# 데이터셋 로드 - 1) 전처리가 완료된 파일이 있으면, 바로 로드
c_filenames = ('data/wip_01.bz2', '/content/drive/MyDrive/해양콘 데이터/rtu_all.bz2')
if os.path.isfile(c_filenames[1]):
    rtu_all_df = pd.read_csv(c_filenames[1], dtype=coldtypes, parse_dates=['일시'])
elif os.path.isfile(c_filenames[0]):
    # 2) 전처리가 완료된 파일은 없지만 중간 파일은 있으면, 완료하고 파일(rtu_all.bz2) 생성
    rtu_air_df, rtu_sea_df = None, None
    chunks = pd.read_csv(c_filenames[0], dtype=coldtypes, parse_dates=['일시'], chunksize=60_000)
    for i, chunk_df in enumerate(chunks):
        # 실시간어장환경조사의 수심(m) 데이터가 0보다 작거나 NaN이면 제외
        cond = (chunk_df['project_name']=='실시간어장환경조사') & ~(chunk_df['수심(m)'] >= 0.0)
        chunk_df = chunk_df[~cond]
        # 해양기상부이의 수심(m) 데이터는 2017년까지는 NaN이고 2018년부터 0.0이므로 모두 0.0으로 대체
        cond = (chunk_df['project_name']=='해양기상부이') & (chunk_df['일시'].dt.year < 2018)
        copied_df = chunk_df.copy()
        copied_df.loc[cond, '수심(m)'] = 0.0
        chunk_df = copied_df.copy()
        # 해양환경측정망의 수심(m) 데이터가 NaN이면 제외 -> 워닝 없음
        cond = (chunk_df['project_name']=='해양환경측정망') & (chunk_df['수심(m)'].isna())
        chunk_df = chunk_df[~cond]
        # 먼 바다 제외
        chunk_df = chunk_df[(32.9 < chunk_df['위도']) & (chunk_df['위도'] < 39.0) & (124.5 < chunk_df['경도']) & (chunk_df['경도'] < 132.5)].reset_index(drop=True)
        # 수심(m)값이 있으면 _sea에, 없으면 _air에
        chunk_air_df = chunk_df[chunk_df['수심(m)'].isna()].groupby(['일시', '위도', '경도']).mean().reset_index()
        chunk_sea_df = chunk_df[chunk_df['수심(m)'].notna()].groupby(['일시', '위도', '경도', '수심(m)']).mean().reset_index()
        # 붙이기
        rtu_air_df = pd.concat([rtu_air_df, chunk_air_df], ignore_index=True) if isinstance(rtu_air_df, pd.DataFrame) else chunk_air_df
        rtu_sea_df = pd.concat([rtu_sea_df, chunk_sea_df], ignore_index=True) if isinstance(rtu_sea_df, pd.DataFrame) else chunk_sea_df

    rtu_air_df = rtu_air_df.groupby(['일시', '위도', '경도']).mean().reset_index()
    rtu_sea_df = rtu_sea_df.groupby(['일시', '위도', '경도', '수심(m)']).mean().reset_index()
    rtu_air_df['육해공'], rtu_sea_df['육해공'] = '공', '해'
    rtu_all_df = pd.concat([rtu_air_df, rtu_sea_df], ignore_index=True).dropna(axis=1, how='all')

    # 전처리 - 파생변수 생성
    stations_df = pd.read_excel('data/raw/정선해양관측소.xls', header=1, usecols=[0, 4, 5], names=['해역', '위도', '경도']).replace('동해COAST', '동해')
    stations_df.drop(stations_df[stations_df['해역'] == '동중국해'].index, inplace=True)
    rtu_all_df['해역'] = predict_by_knn(stations_df[['위도', '경도']], stations_df['해역'], rtu_all_df[['위도', '경도']])
    rtu_all_df['구간'] = '[' + rtu_all_df['위도'].floordiv(0.1).div(10).astype(str) + ', ' +    \
                         rtu_all_df['경도'].floordiv(0.1).div(10).astype(str) + ' ~ ' +         \
                         rtu_all_df['위도'].add(0.1).floordiv(0.1).div(10).astype(str) + ', ' + \
                         rtu_all_df['경도'].add(0.1).floordiv(0.1).div(10).astype(str) + ')'
    rtu_all_df['구간중심_위도'] = rtu_all_df['위도'] // 1 + 0.5
    rtu_all_df['구간중심_경도'] = rtu_all_df['경도'] // 1 + 0.5
    rtu_all_df['연도'] = rtu_all_df['일시'].dt.year
    rtu_all_df['월'] = rtu_all_df['일시'].dt.month
    rtu_all_df['계절'] = np.floor((rtu_all_df['월'] - 3) / 3).replace({0.0: '봄', 1.0: '여름', 2.0: '가을', 3.0: '겨울', -1.0: '겨울'})
    rtu_all_df = rtu_all_df.astype({c: coldtypes[c] for c in coldtypes.keys() if c in rtu_all_df.columns})
    rtu_all_df.to_csv(c_filenames[1], chunksize=50_000, encoding='utf-8', index=False)
else :
    # 3) 전처리를 처음 실행하면, raw 데이터로 부터 중간 파일(wip_01.bz2) 생성
    rtu_all_df = None
    raw_filenames = glob.glob('data/raw/*.zip')

    for raw_filename in raw_filenames:
        chunks = pd.read_csv(raw_filename, encoding='cp949', skiprows=26, dtype=coldtypes, parse_dates=['yyyy-mm-dd hh:mm:ss'], chunksize=60_000)
        for chunk_df in chunks:
            # 컬럼명 변경
            mapper_colnames = {col: col[:-6] for col in chunk_df.columns if col.endswith('_code[null]')}
            chunk_df.rename(columns=mapper_colnames, inplace=True)
            # _qc가 1~2가 아니면 NaN 처리
            qc_cols = [(idx, col) for idx, col in enumerate(chunk_df.columns) if col.endswith('_qc')]
            for idx, col in qc_cols:
                obs_df = chunk_df.iloc[:, [idx-1, idx]].copy()
                obs_df.iloc[~obs_df.iloc[:, 1].isin([1, 2]), 0] = np.nan
                chunk_df[chunk_df.columns[idx-1]] = obs_df.iloc[:, 0]
            # _qc 컬럼 제거
            qc_cols = [col for col in chunk_df.columns if col.endswith('_qc')]
            chunk_df.drop(columns=qc_cols, inplace=True)
            # 데이터 붙이기
            rtu_all_df = pd.concat([rtu_all_df, chunk_df], ignore_index=True) if isinstance(rtu_all_df, pd.DataFrame) else chunk_df

    # 파일 출력
    rtu_all_df.rename(columns=colnames).to_csv(c_filenames[0], chunksize=50_000, encoding='utf-8', index=False)
    print(c_filenames[0], '생성 완료! 전처리를 계속하려면 현재 셀을 재실행 해주세요.')

In [ ]:
rtu_all_df

,일시,위도,경도,bot.depth[m],수심(m),dep_flag,염분[psu],수온[℃],용존산소[mg/L],규산염[μM],아질산성질소[μM],인산염인[μM],질산성질소[μM],투명도[m],기압[hPa],풍속[m/s],건구기온[℃],기압,건구기온,습구기온[℃],수소이온농도[무단위],습구기온,평균파고[m],최대파고[m],풍향[°],돌풍[m/s],유의파고[m],파주기[sec],파향[°],기온[℃],상대습도[%],총질소[μg/L],클로로필-a[μg/L],부유물질 농도[mg/L],규산염[μg/L],아질산성질소[μg/L],인산염인[μg/L],질산성질소[μg/L],용존무기질소[μg/L],화학적산소요구량[mg/L],암모니아성 질소[μg/L],총인[μg/L],부유물질 농도[μg/L],유분[mg/L],풍력등급_code[m/s],유속[cm/s],유향[°],전체수심(m),용존산소[ml/L],육해공,해역,구간,연도,월,계절
0,1997-01-01 00:00:00,33.1125,126.2722,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[33.1, 126.2 ~ 33.2, 126.3)",1997,1,겨울
1,1997-01-01 00:00:00,33.4861,126.9683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[33.4, 126.9 ~ 33.5, 127.0)",1997,1,겨울
2,1997-01-01 00:00:00,34.0083,127.3208,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[34.0, 127.3 ~ 34.1, 127.4)",1997,1,겨울
3,1997-01-01 00:00:00,34.0889,126.6025,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[34.0, 126.6 ~ 34.1, 126.7)",1997,1,겨울
4,1997-01-01 00:00:00,34.0917,125.1014,0.0,NaN,NaN,NaN,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,서해,"[34.0, 125.1 ~ 34.1, 125.2)",1997,1,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10607096,2021-08-31 23:30:00,37.7990,128.9492,NaN,2.0,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607097,2021-08-31 23:30:00,37.7990,128.9492,NaN,3.0,NaN,NaN,19.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607098,2021-08-31 23:30:00,37.7990,128.9492,NaN,5.0,NaN,NaN,22.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607099,2021-08-31 23:30:00,37.9505,124.7295,NaN,3.0,NaN,NaN,21.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,서해,"[37.9, 124.7 ~ 38.0, 124.8)",2021,8,여름


In [ ]:
all_workdata=rtu_all_df.copy()

In [ ]:
all_workdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10607101 entries, 0 to 10607100
Data columns (total 55 columns):
 #   Column          Dtype         
---  ------          -----         
 0   일시              datetime64[ns]
 1   위도              float64       
 2   경도              float64       
 3   bot.depth[m]    float64       
 4   수심(m)           float64       
 5   dep_flag        float64       
 6   염분[psu]         float64       
 7   수온[℃]           float64       
 8   용존산소[mg/L]      float64       
 9   규산염[μM]         float64       
 10  아질산성질소[μM]      float64       
 11  인산염인[μM]        float64       
 12  질산성질소[μM]       float64       
 13  투명도[m]          float64       
 14  기압[hPa]         float64       
 15  풍속[m/s]         float64       
 16  건구기온[℃]         float64       
 17  기압              float64       
 18  건구기온            float64       
 19  습구기온[℃]         float64       
 20  수소이온농도[무단위]     float64       
 21  습구기온            float64       
 22  평균파고[m]         

In [ ]:
all_workdata

,일시,위도,경도,bot.depth[m],수심(m),dep_flag,염분[psu],수온[℃],용존산소[mg/L],규산염[μM],아질산성질소[μM],인산염인[μM],질산성질소[μM],투명도[m],기압[hPa],풍속[m/s],건구기온[℃],기압,건구기온,습구기온[℃],수소이온농도[무단위],습구기온,평균파고[m],최대파고[m],풍향[°],돌풍[m/s],유의파고[m],파주기[sec],파향[°],기온[℃],상대습도[%],총질소[μg/L],클로로필-a[μg/L],부유물질 농도[mg/L],규산염[μg/L],아질산성질소[μg/L],인산염인[μg/L],질산성질소[μg/L],용존무기질소[μg/L],화학적산소요구량[mg/L],암모니아성 질소[μg/L],총인[μg/L],부유물질 농도[μg/L],유분[mg/L],풍력등급_code[m/s],유속[cm/s],유향[°],전체수심(m),용존산소[ml/L],육해공,해역,구간,연도,월,계절
0,1997-01-01 00:00:00,33.1125,126.2722,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[33.1, 126.2 ~ 33.2, 126.3)",1997,1,겨울
1,1997-01-01 00:00:00,33.4861,126.9683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[33.4, 126.9 ~ 33.5, 127.0)",1997,1,겨울
2,1997-01-01 00:00:00,34.0083,127.3208,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[34.0, 127.3 ~ 34.1, 127.4)",1997,1,겨울
3,1997-01-01 00:00:00,34.0889,126.6025,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,남해,"[34.0, 126.6 ~ 34.1, 126.7)",1997,1,겨울
4,1997-01-01 00:00:00,34.0917,125.1014,0.0,NaN,NaN,NaN,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공,서해,"[34.0, 125.1 ~ 34.1, 125.2)",1997,1,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10607096,2021-08-31 23:30:00,37.7990,128.9492,NaN,2.0,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607097,2021-08-31 23:30:00,37.7990,128.9492,NaN,3.0,NaN,NaN,19.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607098,2021-08-31 23:30:00,37.7990,128.9492,NaN,5.0,NaN,NaN,22.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,동해,"[37.7, 128.9 ~ 37.8, 129.0)",2021,8,여름
10607099,2021-08-31 23:30:00,37.9505,124.7295,NaN,3.0,NaN,NaN,21.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해,서해,"[37.9, 124.7 ~ 38.0, 124.8)",2021,8,여름


In [ ]:
sea_forest_1=pd.read_csv("/content/drive/MyDrive/해양콘 데이터/바다숲_1.csv")

In [ ]:
sea_forest_2=pd.read_csv("/content/drive/MyDrive/해양콘 데이터/바다숲_2.csv")

In [ ]:
sea_forest_1

,해역,년도,광역시도,지선,면적,위도,경도
0,동해,2009,강원도,강릉시 안현동 사근진,40.00,37.49532,128.53009
1,동해,2009,강원도,강릉시 안현동 사근진,40.00,37.49691,128.53267
2,동해,2009,강원도,강릉시 안현동 사근진,40.00,37.49146,128.53493
3,동해,2009,강원도,강릉시 안현동 사근진,40.00,37.49323,128.53736
4,동해,2009,경상북도,포항시 남구 신창2리,35.00,35.52961,129.31795
...,...,...,...,...,...,...,...
1227,제주,2020,제주도,제주시 화북동,161.05,33.31500,126.34500
1228,제주,2020,제주도,서귀포시 강정동,161.62,33.13500,126.27000
1229,제주,2020,제주도,서귀포시 강정동,161.62,33.14250,126.27000
1230,제주,2020,제주도,서귀포시 강정동,161.62,33.13500,126.27750


In [ ]:
sea_forest_2

,사업명,년도,시도,시군구,읍면동,통리,어초명,수량,위도,경도,해역,지선
0,동해 바다숲 조성관리사업,2009,강원,강릉,강릉,사근진,46.복합형해중림초,6,37.49532,128.53009,동해,강릉사근진
1,동해 바다숲 조성관리사업,2009,강원,강릉,강릉,사근진,42.터널형어초,74,37.49691,128.53267,동해,강릉사근진
2,동해 바다숲 조성관리사업,2009,강원,강릉,강릉,사근진,29.아치형어초,40,37.49146,128.53493,동해,강릉사근진
3,동해 바다숲 조성관리사업,2009,강원,강릉,강릉,사근진,62.육각패널H빔어초,12,37.49323,128.53736,동해,강릉사근진
4,동해 바다숲 조성관리사업,2009,경북,포항,포항,신창2리,40.십자형해중림초,130,35.52961,129.31795,동해,포항신창2리
...,...,...,...,...,...,...,...,...,...,...,...,...
383,제주 바다숲 조성관리사업,2018,제주,제주시,구좌읍,종달리,51.이중돔형어초,57,36.18584,129.22786,제주,구좌읍종달리
384,제주 바다숲 조성관리사업,2018,제주,서귀포시,성산읍,삼달·신산,73.다공질이식형해중림초,53,36.18622,129.22960,제주,성산읍삼달·신산
385,제주 바다숲 조성관리사업,2018,제주,서귀포시,성산읍,삼달·신산,78.십자주름초,67,36.17765,129.23052,제주,성산읍삼달·신산
386,제주 바다숲 조성관리사업,2018,제주,서귀포시,성산읍,신천,78.십자주름초,134,35.05814,129.07210,제주,성산읍신천


In [ ]:
import missingno as msno

In [ ]:
# msno.bar(all_workdata)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# sns.heatmap(all_workdata.isna(),cbar=False)

In [ ]:
# pd.read_csv("/content/drive/MyDrive/해양콘 데이터/인공어초_기초자료_해조류/2. (제공자료1) 2009-2020 바다숲 인공어초 시설현황(사업비, 해조류 종류).xlsx")

In [ ]:
all_workdata.isna().sum()

일시                       0
위도                       0
경도                       0
bot.depth[m]       2670975
수심(m)               371024
dep_flag          10289120
염분[psu]            9904508
수온[℃]               141342
용존산소[mg/L]         9825895
규산염[μM]           10528595
아질산성질소[μM]        10538727
인산염인[μM]          10524325
질산성질소[μM]         10548637
투명도[m]            10525312
기압[hPa]            8904987
풍속[m/s]            8963827
건구기온[℃]           10583747
기압                10560398
건구기온              10559744
습구기온[℃]           10607099
수소이온농도[무단위]       10525680
습구기온              10601388
평균파고[m]            9117854
최대파고[m]            8958930
풍향[°]              8991646
돌풍[m/s]            8987481
유의파고[m]            8960804
파주기[sec]           8974543
파향[°]              8958900
기온[℃]              9227416
상대습도[%]            9433065
총질소[μg/L]         10555332
클로로필-a[μg/L]      10446554
부유물질 농도[mg/L]     10594336
규산염[μg/L]         10568380
아질산성질소[μg/L]      10547071
인산염인[μg/L]        10555001
질

In [ ]:
all_workdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10607101 entries, 0 to 10607100
Data columns (total 55 columns):
 #   Column          Dtype         
---  ------          -----         
 0   일시              datetime64[ns]
 1   위도              float64       
 2   경도              float64       
 3   bot.depth[m]    float64       
 4   수심(m)           float64       
 5   dep_flag        float64       
 6   염분[psu]         float64       
 7   수온[℃]           float64       
 8   용존산소[mg/L]      float64       
 9   규산염[μM]         float64       
 10  아질산성질소[μM]      float64       
 11  인산염인[μM]        float64       
 12  질산성질소[μM]       float64       
 13  투명도[m]          float64       
 14  기압[hPa]         float64       
 15  풍속[m/s]         float64       
 16  건구기온[℃]         float64       
 17  기압              float64       
 18  건구기온            float64       
 19  습구기온[℃]         float64       
 20  수소이온농도[무단위]     float64       
 21  습구기온            float64       
 22  평균파고[m]         

In [ ]:
seaweed_1=pd.read_excel("/content/drive/MyDrive/해양콘 데이터/인공어초_기초자료_해조류/2009-2020_바다숲_해조.xlsx",sheet_name = None)

In [ ]:
seaweed_1.keys()

dict_keys(['동해 바다숲', '서해 바다숲', '남해 바다숲', '제주 바다숲'])

In [ ]:
seaweed_2=pd.read_excel("/content/drive/MyDrive/해양콘 데이터/인공어초_기초자료_해조류/2009-2020_인공어초_해조.xlsx")

In [ ]:
seaweed_1

{'남해 바다숲':     번호            사업명  사업년도  해역  ...             어초명 사업비(천원) 이식 해조류   비고
 0    1  남해 바다숲 조성관리사업  2009  남해  ...       49.다기능성어초     NaN    NaN  NaN
 1    2  남해 바다숲 조성관리사업  2009  남해  ...       32.반톱니형어초     NaN    NaN  NaN
 2    3  남해 바다숲 조성관리사업  2009  남해  ...    37.사각복합형인공어초     NaN    NaN  NaN
 3    4  남해 바다숲 조성관리사업  2009  남해  ...        42.터널형어초     NaN    NaN  NaN
 4    5  남해 바다숲 조성관리사업  2010  남해  ...        9.방갈로형어초     NaN    NaN  NaN
 ..  ..            ...   ...  ..  ...             ...     ...    ...  ...
 75  76  남해 바다숲 조성관리사업  2019  남해  ...  85.다층식피라미드인공어초  389000     감태  NaN
 76  77  남해 바다숲 조성관리사업  2020  남해  ...      88.사단경사형어초  200000     곰피  NaN
 77  78  남해 바다숲 조성관리사업  2020  남해  ...        9.방갈로형어초  400000     곰피  NaN
 78  79  남해 바다숲 조성관리사업  2020  남해  ...  85.다층식피라미드인공어초  323895     감태  NaN
 79  80  남해 바다숲 조성관리사업  2020  남해  ...   73.다공질이식형해중림초  245895     감태  NaN
 
 [80 rows x 12 columns],
 '동해 바다숲':         번호            사업명  사업년도  해역  ... 사업비(천원) 이식 해조류   비고    

In [ ]:
seaweed_1=pd.concat(seaweed_1, ignore_index=True)

In [ ]:
seaweed_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           374 non-null    float64
 1   사업명          382 non-null    object 
 2   사업년도         382 non-null    int64  
 3   해역           382 non-null    object 
 4   시도           382 non-null    object 
 5   시군구          382 non-null    object 
 6   읍면동          237 non-null    object 
 7   통리           352 non-null    object 
 8   어초명          382 non-null    object 
 9   수량           273 non-null    float64
 10  사업비(천원)      371 non-null    object 
 11  이식 해조류       371 non-null    object 
 12  비고           13 non-null     object 
 13  Unnamed: 13  1 non-null      object 
dtypes: float64(2), int64(1), object(11)
memory usage: 41.9+ KB


In [ ]:
seaweed_1["사업명"].value_counts()

동해 바다숲 조성관리사업    185
제주 바다숲 조성관리사업     88
남해 바다숲 조성관리사업     80
서해 바다숲 조성관리사업     29
Name: 사업명, dtype: int64

In [ ]:
seaweed_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      847 non-null    int64  
 1   사업명     847 non-null    object 
 2   설치년도    847 non-null    int64  
 3   인공어초명   385 non-null    object 
 4   설치개수    385 non-null    float64
 5   설치비용    384 non-null    float64
 6   위도(N)   847 non-null    object 
 7   경도(E)   847 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 53.1+ KB


In [ ]:
seaweed_2.tail(50)

,번호,사업명,설치년도,인공어초명,설치개수,설치비용,위도(N),경도(E)
797,798,태안군 중부 연안바다목장,2018,NaN,NaN,NaN,36° 39.123′,126° 07.342′
798,799,군산시(옥도) 연안바다목장,2018,21.대형강제어초,2.0,274066.000,35° 49.568′,126° 21.880′
799,800,군산시(옥도) 연안바다목장,2018,NaN,NaN,NaN,35° 49.565′,126° 21.878′
800,801,고창군 연안바다목장,2018,67.개량요철형어초,59.0,295000.000,35° 31.196′,126° 24.948′
801,802,고창군 연안바다목장,2018,NaN,NaN,NaN,35° 31.238′,126° 24.988′
802,803,삼척(임원) 연안바다목장,2018,65.미로형인공어초,45.0,209700.000,37˚ 13.785′,129˚ 21.063′
803,804,삼척(임원) 연안바다목장,2018,54.날개부를가진어초,40.0,224000.000,37˚ 13.757′,129˚ 21.071′
804,805,삼척(임원) 연안바다목장,2018,62.육각패널에이치빔어초,23.0,203550.000,37˚ 13.795′,129˚ 21.063′
805,806,삼척(초곡) 연안바다목장,2018,44.부채꼴베란다사각어초,50.0,230000.000,37˚ 18.750′,129˚ 17.602′
806,807,삼척(초곡) 연안바다목장,2018,65.미로형인공어초,58.0,273760.000,37˚ 18.760′,129˚ 17.631′


In [ ]:
seaweed_2.to_csv("바다숲_해조_2.csv",index=False)

In [ ]:
seaweed_1.to_csv("바다숲_해조_1.csv",index=False)

In [ ]:
seaweed_1


,번호,사업명,사업년도,해역,시도,시군구,읍면동,통리,어초명,수량,사업비(천원),이식 해조류,비고,Unnamed: 13
0,1.0,동해 바다숲 조성관리사업,2009,동해,강원,강릉,NaN,사근진,46.복합형해중림초,6.0,0,0,NaN,NaN
1,2.0,동해 바다숲 조성관리사업,2009,동해,강원,강릉,NaN,사근진,42.터널형어초,74.0,0,0,NaN,NaN
2,3.0,동해 바다숲 조성관리사업,2009,동해,강원,강릉,NaN,사근진,29.아치형어초,40.0,0,0,NaN,NaN
3,4.0,동해 바다숲 조성관리사업,2009,동해,강원,강릉,NaN,사근진,62.육각패널H빔어초,12.0,0,0,NaN,보식된 해조류 양식은 안넣음.
4,5.0,동해 바다숲 조성관리사업,2009,동해,경북,포항,NaN,신창2리,40.십자형해중림초,130.0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,92.0,제주 바다숲 조성관리사업,2018,제주,제주,제주시,구좌읍,종달리,51.이중돔형어초,57.0,306192,감태,NaN,NaN
378,93.0,제주 바다숲 조성관리사업,2018,제주,제주,서귀포시,성산읍,삼달·신산,73.다공질이식형해중림초,53.0,285200,감태,NaN,NaN
379,94.0,제주 바다숲 조성관리사업,2018,제주,제주,서귀포시,성산읍,삼달·신산,78.십자주름초,67.0,271600,감태,NaN,NaN
380,95.0,제주 바다숲 조성관리사업,2018,제주,제주,서귀포시,성산읍,신천,78.십자주름초,134.0,271600,감태,NaN,NaN


In [ ]:
seaweed_2

,번호,사업명,설치년도,인공어초명,설치개수,설치비용,위도(N),경도(E)
0,1,영덕군 연안바다목장,2011,30.원통2단강제어초,2.0,208750.0,36° 33.501′,129° 26.601′
1,2,영덕군 연안바다목장,2011,NaN,NaN,NaN,36° 33.535′,129° 26.601′
2,3,영덕군 연안바다목장,2011,1.사각형어초,200.0,123550.0,36° 33.505′,129° 26.625′
3,4,영덕군 연안바다목장,2011,NaN,NaN,NaN,36° 33.505′,129° 26.655′
4,5,영덕군 연안바다목장,2011,NaN,NaN,NaN,36° 33.530′,129° 26.655′
...,...,...,...,...,...,...,...,...
842,843,삼척(임원) 연안바다목장,2020,88.사단경사형어초,66.0,238812.0,-,-
843,844,삼척(초곡) 연안바다목장,2020,31.정삼각뿔형어초,68.0,296643.0,-,-
844,845,삼척(초곡) 연안바다목장,2020,44.부채꼴베란다사각어초,50.0,298259.0,-,-
845,846,태안군(북부) 연안바다목장,2020,70.트리톤A형어초,151.0,291839.0,-,-


In [ ]:
# 데이터를 연, 월로 그룹바이해서 데이터 한번 살펴보기, 시각화
#  하나는 연단위, 하나응 연,월 합친것 그룹바이

In [ ]:
work_year=all_workdata.groupby("연도").mean().reset_index()
work_year

,연도,위도,경도,bot.depth[m],수심(m),dep_flag,염분[psu],수온[℃],용존산소[mg/L],규산염[μM],아질산성질소[μM],인산염인[μM],질산성질소[μM],투명도[m],기압[hPa],풍속[m/s],건구기온[℃],기압,건구기온,습구기온[℃],수소이온농도[무단위],습구기온,평균파고[m],최대파고[m],풍향[°],돌풍[m/s],유의파고[m],파주기[sec],파향[°],기온[℃],상대습도[%],총질소[μg/L],클로로필-a[μg/L],부유물질 농도[mg/L],규산염[μg/L],아질산성질소[μg/L],인산염인[μg/L],질산성질소[μg/L],용존무기질소[μg/L],화학적산소요구량[mg/L],암모니아성 질소[μg/L],총인[μg/L],부유물질 농도[μg/L],유분[mg/L],풍력등급_code[m/s],유속[cm/s],유향[°],전체수심(m),용존산소[ml/L]
0,1961,36.447055,129.516918,0.0,80.383015,NaN,33.739115,13.326936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1017.556404,19.514685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1962,35.594756,128.572663,0.0,64.190918,NaN,33.758325,12.488468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1020.689613,16.317944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1963,35.291191,128.332151,0.0,53.754295,NaN,33.521930,12.188490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024.254676,17.868222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964,35.492999,128.348672,0.0,59.213529,NaN,33.668215,13.518049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.414145,17.290239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1965,35.634766,128.558692,0.0,69.399346,NaN,33.823158,13.037993,5.430080,NaN,NaN,0.520257,NaN,13.005469,NaN,NaN,NaN,1016.695719,16.183866,NaN,7.863893,23.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,35.460138,127.647964,0.0,4.607083,0.0,33.218550,17.085379,6.356754,9.070305,0.186853,0.408905,6.173673,9.636586,1016.527394,5.652269,NaN,NaN,NaN,NaN,8.147399,NaN,0.720047,1.653526,187.314431,7.620054,1.007377,5.523268,192.322675,15.303675,73.253567,264.975726,2.088105,15.901927,66.525958,7.468102,2.279545,40.884552,117.442201,1.559569,20.527719,27.792759,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,2018,35.461239,127.650723,0.0,4.535699,0.0,33.291436,16.786520,6.293718,8.860007,0.149725,0.407929,5.908944,10.102086,1016.487474,5.678798,NaN,NaN,NaN,NaN,8.148562,NaN,0.732367,1.690128,175.620270,7.690224,1.027564,5.708205,184.388768,10.593088,NaN,262.512844,2.207408,15.565269,78.580798,7.115844,1.991724,39.144813,134.728312,1.699437,37.023156,31.243781,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2019,35.480816,127.678119,0.0,4.550838,0.0,33.184558,17.135474,6.544433,9.008595,0.189696,0.434907,5.777755,9.513333,1016.159381,5.645195,NaN,NaN,NaN,NaN,8.152642,NaN,0.687063,1.597510,183.295954,7.381225,0.964491,5.416891,178.958737,14.837895,NaN,265.013937,2.558052,13.337222,88.297259,5.964000,2.521594,34.911412,113.214219,1.619156,27.464750,29.784594,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2020,35.310798,127.357137,NaN,4.281391,NaN,29.006773,18.356770,7.439361,10.065730,0.184152,0.402782,6.721317,8.505240,1016.508160,5.836081,NaN,NaN,NaN,NaN,8.123141,NaN,0.732486,1.704197,186.356795,7.626994,1.038344,5.702346,182.394759,14.727130,NaN,299.080896,3.617573,16.926686,375.280004,7.998835,14.261263,116.877129,153.652298,1.834692,28.776332,33.447714,NaN,NaN,NaN,NaN,NaN,465.675066,6.300677


In [ ]:
work_year_mo=all_workdata.groupby(["연도","월"]).mean().reset_index()
work_year_mo

,연도,월,위도,경도,bot.depth[m],수심(m),dep_flag,염분[psu],수온[℃],용존산소[mg/L],규산염[μM],아질산성질소[μM],인산염인[μM],질산성질소[μM],투명도[m],기압[hPa],풍속[m/s],건구기온[℃],기압,건구기온,습구기온[℃],수소이온농도[무단위],습구기온,평균파고[m],최대파고[m],풍향[°],돌풍[m/s],유의파고[m],파주기[sec],파향[°],기온[℃],상대습도[%],총질소[μg/L],클로로필-a[μg/L],부유물질 농도[mg/L],규산염[μg/L],아질산성질소[μg/L],인산염인[μg/L],질산성질소[μg/L],용존무기질소[μg/L],화학적산소요구량[mg/L],암모니아성 질소[μg/L],총인[μg/L],부유물질 농도[μg/L],유분[mg/L],풍력등급_code[m/s],유속[cm/s],유향[°],전체수심(m),용존산소[ml/L]
0,1961,1,34.895007,129.259603,0.0,52.241379,NaN,34.232759,13.253103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1019.200000,8.160000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961,10,35.650111,129.879662,0.0,77.209756,NaN,33.807839,15.816488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023.116667,20.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1961,11,37.118986,129.843841,0.0,87.092308,NaN,33.876688,14.218400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1027.275556,16.348889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1961,12,35.885253,128.889842,0.0,71.211215,NaN,33.925515,13.157196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1022.345977,11.740698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1961,2,34.433300,128.757500,0.0,34.000000,NaN,34.630000,12.572000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2021,5,35.277677,127.384985,NaN,3.787517,NaN,30.786621,15.292874,8.468410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.639535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
728,2021,6,35.267577,127.379807,NaN,3.758877,NaN,29.594108,18.841795,7.528755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.392708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
729,2021,7,35.209480,127.295003,NaN,3.622942,NaN,24.073906,22.998679,6.923167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.554255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,2021,8,35.157569,127.192094,NaN,3.464052,NaN,16.606495,25.334622,6.012454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.658889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import plotly.express as px

In [ ]:
import plotly.express as px

fig = px.bar(work_year, x='연도', y='염분[psu]')
fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(work_year_mo, x='연도', y='염분[psu]', barmode='group')
fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(work_year_mo, x='월', y='수심(m)',)
fig.show()